---
## Projectr : 하루시작 지하철 혼잡도 분석 02 
## Description : 
    - 서울 교통공사 역별 일별 시간대별 승하차인원 정보
    [source link] (https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do#)
    
## Author : Forrest Dpark (분석 담당)
## Date : 2024.05.31 ~
## Detail : 
## Update: 
    * 2024.06.02 pdg : 일별 데이터 정제 
        - 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.
    * 2024.06.04 pdg : ppt 만들며 정리 
        - 데이터 정제중 일부 프린트 결과 명확하게 수정함. 
        - 마지막 테이블에 환승역수 관련 칼럼을 추가함. 
    * 2024.06.09 by pdg : 군자, 상봉 검색 오류 이슈 해결 
        - 모듈 대체 할수잇는 코드 대체 
        - 문제파악: station_info 를 만들때 역명 별로 역사코드를 하나씩만 지정되어서 환승역 은 저장이 안됬다. 
        따라서 군자역은 5호선 코드만 저장되고 7호선 코드는 지정되지 않아 나중에 테이블 머지할때 
        사라지게 된것!! 
        또한 상봉역 같은 경우 () 표시가 있었기에 최종테이블 검색 조건이 == 이어서 사라지게된것!!
        - duplicate drop 사용하여 해결. 해결된 함수 모듈화 완료
    * 2024.06.10 by pdg : 08년도 부터 24년까지 데이터 추가 
        - 관찰결과 엑셀데이터인 17,18,19 년도 자료는 칼럼이름이 unnamed 로 나와서 수정해야함. 
        - 모든데이터 받는데 1분 30 초 걸림. 
        - 


In [2]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
from Functions import Service
Service.plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [15]:
# Data loading 
subway_dict= {
'subway23_0' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원_20230630.csv", encoding='euc-kr'),
'subway23_1' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20231031.csv", encoding='euc-kr'),
'subway23_2' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_23.11_24.01.csv", encoding='euc-kr'),
'subway22' : pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20221231.csv", encoding='euc-kr'),
"subway21" : pd.read_csv("../Data/교통공사데이터/서울교통공사 2021년 일별 역별 시간대별 승하차인원(1_8호선).csv", encoding='euc-kr'),
'subway20' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2020년 일별 역별 시간대별 승하차인원(1_8호선).csv", encoding='euc-kr'),
'subway19' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx"),
'subway18' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2018년 일별 역별 시간대별 승하차인원(1_8호선).xlsx"),
'subway17' : pd.read_excel("../Data/교통공사데이터/서울교통공사 2017년 일별 역별 시간대별 승하차인원(1_8호선).xlsx"),
'subway16' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2016년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway15' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2015년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway14' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2014년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway13' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2013년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway12' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2012년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway11' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2011년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway10' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2010년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway09' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2009년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr'),
'subway08' : pd.read_csv("../Data/교통공사데이터/서울교통공사 2008년 일별 역별 시간대별 승하차인원(1~8호선).csv", encoding='euc-kr')
}

def dataInfoPrint(data):
    Service.dataInfoProcessing(data)
    return data
for key in subway_dict.keys():
    subway_dict[key]=dataInfoPrint(subway_dict[key])
    print(f"{key.split('subway')[-1]}년도 데이터 ")
    subway_dict[key].head(3)


1. Data row/colum numbers : 98603/26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			98603/98603 [uniq/raw]
	1.[수송일자(object)]: 			181/98603 [uniq/raw]
	2.[호선(object)]: 			8/98603 [uniq/raw]
	3.[역번호(int64)]: 			279/98603 [uniq/raw]
	4.[역명(object)]: 			244/98603 [uniq/raw]
	5.[승하차구분(object)]: 			2/98603 [uniq/raw]
	6.[06시이전(int64)]: 			1471/98603 [uniq/raw]
	7.[06-07시간대(int64)]: 		2420/98603 [uniq/raw]
	8.[07-08시간대(int64)]: 		5859/98603 [uniq/raw]
	9.[08-09시간대(int64)]: 		8988/98603 [uniq/raw]
	 ...etc (추가로 16개의 칼럼이 있습니다 )
23_0년도 데이터 

1. Data row/colum numbers : 165376/26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			165376/165376 [uniq/raw]
	1.[수송일자(object)]: 			304/165376 [uniq/raw]
	2.[호

In [16]:
subway= subway_dict['subway22']


1. Data row/colum numbers : 199080/26

2. null ceck 결과 총 82346개의 null 이 있습니다!
   => 25번째.[24시이후]컬럼 :  null 82346 개,	 not null 116734 개

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	7.[06-07시간대(int64)]: 		2515/199080 [uniq/raw]
	8.[07-08시간대(int64)]: 		6520/199080 [uniq/raw]
	9.[08-09시간대(int64)]: 		10884/199080 [uniq/raw]
	 ...etc (추가로 16개의 칼럼이 있습니다 )


> 25번째 칼럼인 24시 이후 데이터의 null 은 운행을 하지 않아서 null 인가?

In [15]:
# 관심있는 칼럼 정보만 추출해서 보기 
interesting_colnumbers = [i for i in range(1,7)] + [-2,-1]
print(interesting_colnumbers)
subway[subway['24시이후'].isnull()].iloc[:,interesting_colnumbers].head()

[1, 2, 3, 4, 5, 6, -2, -1]


,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,23-24시간대,24시이후
0,2022-01-01,1,150,서울역,승차,120,234,NaN
1,2022-01-01,1,150,서울역,하차,113,131,NaN
2,2022-01-01,1,151,시청,승차,38,38,NaN
3,2022-01-01,1,151,시청,하차,31,24,NaN
4,2022-01-01,1,152,종각,승차,44,62,NaN


---
#### null check

In [16]:
# null check
havenulldate = subway[subway['24시이후'].isnull()].iloc[:,interesting_colnumbers]
# havenulldate['수송일자'].unique()[[1,2,-2,-1]]
print("** 24 시 이후 운행을하지 않아 null 을 가지고 있는 날짜")
havenull_test= havenulldate['수송일자'].unique()
# 2022 년 1 월 1일부터  5월 31 일까지 는 12시 이후 영업을 안한 모양.. -> 코로나 단축운행!! 
for idx, row in enumerate(havenull_test):
    if idx <1 or idx> len(havenull_test)-2:
        row_print = "\t|-"+row+" ~ " if idx<2 else row
        print(row_print, end="")
else: print(f"\n\t|-총 {len(havenull_test)}개의 row 에서 null 발생")

** 24 시 이후 운행을하지 않아 null 을 가지고 있는 날짜
	|-2022-01-01 ~ 2022-05-31
	|-총 151개의 row 에서 null 발생


> Nan 을 0 으로 대체 

In [17]:

# Null 을 0으로 대체 
subway=Service.dataInfoProcessing(subway,replace_Nan=True,nanFillValue=0)


1. Data row/colum numbers : 199080/26

2. null ceck 결과 총 82346개의 null 이 있습니다!
   => 25번째.[24시이후]컬럼 :  null 82346 개,	 not null 116734 개

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	7.[06-07시간대(int64)]: 		2515/199080 [uniq/raw]
	8.[07-08시간대(int64)]: 		6520/199080 [uniq/raw]
	9.[08-09시간대(int64)]: 		10884/199080 [uniq/raw]
	 ...etc (추가로 16개의 칼럼이 있습니다 )


>  해당 데이터는 8개의 호선이 있고 역번호는 558개, 역명은 244 개가 있다.. 왜?  역번호와 역 명은 1:1 이어야 하는게 아닌가?

In [18]:
## 역명 조회
# 지하철 역 테이블 과 역번호 를 따로 만들어서 관리하도록 하자. csv 저장 필요 
subway_info=Service.subway_info_table(subway, save=True)

subway_info

첫 수송일자 : 2022-01-01
마지막 수송일자 : 2022-12-31
132480번째 데이터   <-value errer: 
[132480]는 값에러 0 으로 대체함


,역사코드,역명,호선,환승역수
0,150,서울역,1,2
1,426,서울역,4,2
2,151,시청,1,2
3,201,시청,2,2
4,152,종각,1,1
...,...,...,...,...
274,2649,신내,6,1
275,2758,상동,7,1
276,2761,부평구청,7,1
277,2753,까치울,7,1


--- 
> 역코드 unique 는 왜 500 개가 넘을까?
> 승차에는 없는 띄어쓰기가 하차에는 있음! 그리고 132480번째 데이터   에는 띄어쓰기가 들어가있음. 

---
###  마지막 수송 일자를 기준으로 역 이름과 역 코드를 판별한다. 

- 2024.06.09 update by pdg 
> 굳이 마지막 일자를 기준으로 가를 필요는 없다. 어차피 역사 코드는  고유함. 그리고 조사결과 있다가 사라지는 역은 존재하지 않는다. 



---
#### 역명과 고유역번호가 일치하는가?

In [19]:
## 역명과 고유역번호가 일치하는가?
print("역명개수:",len(subway_info['역명'].unique()),"고유역번호개수",len(subway_info['역사코드'].unique()))
subway_info['역명'].value_counts()
subway_info[subway_info['역명']=='종로3가'].iloc[:,:].reset_index(drop=True)
# subway[subway['고유역번호(외부역코드)']==153]['호선'].iloc[0]

역명개수: 244 고유역번호개수 279


,역사코드,역명,호선,환승역수
0,153,종로3가,1,3
1,319,종로3가,3,3
2,2535,종로3가,5,3


> 종로 3가 는 1호선, 3호선, 5호선 세 종류의 호선마다 역이 있다.     
> 중복문제를 피하기 위해서 역_이름코드 라는 칼럼을 하나 만들어서 primary key 로 만들자

In [20]:
subway_info

,역사코드,역명,호선,환승역수
0,150,서울역,1,2
1,426,서울역,4,2
2,151,시청,1,2
3,201,시청,2,2
4,152,종각,1,1
...,...,...,...,...
274,2649,신내,6,1
275,2758,상동,7,1
276,2761,부평구청,7,1
277,2753,까치울,7,1


In [21]:
# 역 개수 통계 수치 파악 
subway_info.pivot_table(
    aggfunc=np.size,
    index='호선',
    values='역명'
).T

호선,1,2,3,4,5,6,7,8
역명,10,50,34,26,56,39,46,18


----
#### 지하철 역사의 위도 경도 데이터 추가 